In [ ]:
!pip install seleniumbase 
!pip install selenium

In [23]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

In [46]:
class Leetcode:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.tag_list=[]
        self.companies_list=[]

    def Login(self,id,pw):
        url = "https://leetcode.com/accounts/login/"
        self.driver.get(url)
        id_field = self.driver.find_element(by=By.ID, value="id_login")
        id_field.send_keys(id)
        time.sleep(1)
        pw_field = self.driver.find_element(by=By.ID, value="id_password")
        pw_field.send_keys(pw)
        #no need check ctp
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.ID, "signin_btn"))).click()
        time.sleep(5)
        self.GetAllTagAndCompaniesList()

    def GetAllTagAndCompaniesList(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//button[@id='headlessui-popover-button-:r5:'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-blue dark:text-dark-blue m-1 flex cursor-pointer items-center px-1'][normalize-space()='Expand'])[1]"))).click()
        tag_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[1]")
        tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('tag.txt', 'w') as f:
            for e in tag_elements:
                self.tag_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-sm leading-5 font-normal text-label-2 dark:text-dark-label-2'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Expand')])[1]"))).click()
        companies_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[2]")
        companies_elements = companies_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('companies.txt', 'w') as f:
            for e in companies_elements:
                self.companies_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        
        

In [47]:
test = Leetcode()
id = "ng_min_teck@hotmail.com"
pw ="NUS-ISS-MTECH-IS"
test.Login(id,pw)
